Chuẩn bị dữ liệu

In [14]:
comment_df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("comments2.csv")
comment_df

DataFrame[video_id: string, author: string, comment_text: string, published_at: string, like_count: string]

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Tạo SparkSession
spark = SparkSession.builder.appName("SentimentAnalysis").getOrCreate()

# Đọc dữ liệu
data = spark.read.csv("comments2.csv", header=True, inferSchema=True)

# Hiển thị thông tin dữ liệu
data.printSchema()
data.show(5)

# Loại bỏ giá trị NULL trong cột comment_text
data = data.filter(col("comment_text").isNotNull())

root
 |-- video_id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- published_at: string (nullable = true)
 |-- like_count: string (nullable = true)

+-----------+------------------+--------------------+--------------------+----------+
|   video_id|            author|        comment_text|        published_at|like_count|
+-----------+------------------+--------------------+--------------------+----------+
|N-gpD9QqTK0|    @truchothi7758|Sau khi dc xem vi...|2024-12-10T15:26:42Z|         0|
|N-gpD9QqTK0|     @ÁiNguyễn-b1p|Cảm ơn khoai.tất ...|2024-12-10T15:15:36Z|         0|
|N-gpD9QqTK0| @UyenNguyen-lu5bu|Thích tập này quá...|2024-12-10T14:42:05Z|         0|
|N-gpD9QqTK0|@dieunguyeninh3741|Xem video của Kho...|2024-12-10T14:33:42Z|         0|
|N-gpD9QqTK0|  @huynhthuytram24|Một video thực sự...|2024-12-10T13:54:28Z|         0|
+-----------+------------------+--------------------+--------------------+----------+
only showing

Tiền xử lí

In [4]:
# Lọc các hàng có video_id không hợp lệ
data = data.filter(col("video_id").rlike("^[a-zA-Z0-9_-]{11}$"))
data.select("video_id").distinct().show(truncate=False)

+-----------+
|video_id   |
+-----------+
|aoFeasRsiyk|
|oL9OX4e2Kl4|
|NTEgu8uWS1I|
|M_WD9Dxayk8|
|IJ24VtLd1jU|
|N-gpD9QqTK0|
|NgaZHtCr_a0|
|92-IbWKp_3k|
|cJD4fc5l3fM|
|mzeh6lWhFi4|
|0Q8fkicguR8|
|MLBhMV8k6e0|
|5cW2Kj4_7HY|
|7TdSEX4JFdw|
|xEnYZ55FCG8|
|E-JMv4ebt18|
|7v7EBziokmM|
|FDMDKktG75I|
|5AJd2FJUVkc|
|DRo7i4jQ3Z8|
+-----------+
only showing top 20 rows



In [5]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

# Tokenizer: tách từ
tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")
data = tokenizer.transform(data)

# Loại bỏ từ dừng
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
data = stopwords_remover.transform(data)

# HashingTF: Tạo vector từ văn bản
hashing_tf = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=1000)
data = hashing_tf.transform(data)

# IDF: Chuẩn hóa vector
idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(data)
data = idf_model.transform(data)

data.select("comment_text", "filtered_words", "features").show(5)


+--------------------+--------------------+--------------------+
|        comment_text|      filtered_words|            features|
+--------------------+--------------------+--------------------+
|Sau khi dc xem vi...|[sau, khi, dc, xe...|(1000,[28,34,72,7...|
|Cảm ơn khoai.tất ...|[cảm, ơn, khoai.t...|(1000,[92,153,177...|
|Thích tập này quá...|[thích, tập, này,...|(1000,[61,244,376...|
|Xem video của Kho...|[xem, video, của,...|(1000,[34,92,317,...|
|Một video thực sự...|[một, video, thực...|(1000,[28,67,76,9...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



Huấn luyện mô hình

In [6]:
from pyspark.sql.functions import when

data = data.withColumn(
    "label",
    when(col("comment_text").rlike("cảm ơn|tuyệt vời|yêu thích|hay|xuất sắc|đỉnh cao|tốt nhất|dễ thương|tuyệt|ý nghĩa|ấm áp|cười|chúc mừng"), 2)  # Tích cực
    .when(col("comment_text").rlike("ổn|bình thường|cũng được|ok|tạm ổn"), 1)  # Trung lập
    .when(col("comment_text").rlike("tệ|kém|không thích|chán|thất vọng|quá tệ|dở|kém chất lượng|dở tệ|qc|quảng cáo"), 0)  # Tiêu cực
    .otherwise(1)  # Gán mặc định là trung lập nếu không khớp
)
data.select("comment_text", "label").show(10)

+--------------------+-----+
|        comment_text|label|
+--------------------+-----+
|Sau khi dc xem vi...|    2|
|Cảm ơn khoai.tất ...|    2|
|Thích tập này quá...|    2|
|Xem video của Kho...|    2|
|Một video thực sự...|    2|
|Anh Khoai dễ thươ...|    2|
|Cảm ơn vì những g...|    1|
|người ta ổ chuột ...|    1|
|em theo dõi hành ...|    1|
|        QC nhiều quá|    1|
+--------------------+-----+
only showing top 10 rows



In [7]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Thay các giá trị không hợp lệ trong cột like_count bằng 0
data = data.withColumn(
    "like_count", 
    when(col("like_count").cast("int").isNotNull(), col("like_count").cast("int")).otherwise(0)
)

# Kết hợp features và like_count
assembler = VectorAssembler(inputCols=["features", "like_count"], outputCol="final_features")
data = assembler.transform(data)

# Chia tập dữ liệu thành train và test
train, test = data.randomSplit([0.8, 0.2], seed=42)

# Logistic Regression
lr = LogisticRegression(featuresCol="final_features", labelCol="label")
model = lr.fit(train)

# Dự đoán trên tập test
predictions = model.transform(test)
predictions.select("comment_text", "label", "prediction").show(10)

# Đánh giá mô hình
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Độ chính xác: {accuracy}")

+--------------------+-----+----------+
|        comment_text|label|prediction|
+--------------------+-----+----------+
|Xa xôi vậy mà BYD...|    1|       1.0|
|Khoai đi nhớ có t...|    1|       2.0|
|Thú vị thiệt luôn...|    1|       1.0|
|"DUNG TU "" MOI N...|    1|       1.0|
|Chúc anh Khoai th...|    1|       1.0|
|không mấy ngày an...|    1|       1.0|
|Mất tích cả tháng...|    1|       1.0|
|Hời ơi ổng đăng v...|    1|       1.0|
|nhìn anh Phương d...|    1|       1.0|
|Hải quan VN mình ...|    1|       1.0|
+--------------------+-----+----------+
only showing top 10 rows

Độ chính xác: 0.9257504594649786


Phân tích xu hướng bình luận trên video

In [ ]:
from pyspark.sql.functions import count, mean

# Tính số lượng và tỷ lệ cảm xúc trên mỗi video
video_sentiment = predictions.groupBy("video_id", "prediction").agg(count("prediction").alias("count"))
video_sentiment = video_sentiment.groupBy("video_id").pivot("prediction").sum("count").fillna(0)

# Đổi tên cột cảm xúc
video_sentiment = video_sentiment.withColumnRenamed("negative", "0")\
                                 .withColumnRenamed("neutral", "1")\
                                 .withColumnRenamed("positive", "2")

# Hiển thị kết quả
video_sentiment.show()

+-----------+---+---+---+
|   video_id|0.0|1.0|2.0|
+-----------+---+---+---+
|mLmPajA_3Gg|  0|220|110|
|fw02hy6vqaQ|  2| 93| 12|
|DRo7i4jQ3Z8|  1|126| 14|
|OexdHszpU00|  1|124| 27|
|aoFeasRsiyk|  0|246| 67|
|KMdlKttaWJY|  1| 72|  7|
|GOz2qHVKKC8|  0|281| 43|
|4NWNhzcRRtg|  0|195| 24|
|rcgrVgVlV5c|  2|125|  9|
|2gVWXkHSpL0|  1|119| 16|
|SjtMbfXcc6U|  1|406| 78|
|28oGGAiNtrU|  2|259| 38|
|oL9OX4e2Kl4|  3|371| 85|
|NTEgu8uWS1I|  2|263| 43|
|QswEGJs4a74|  0|195| 32|
|Psj74U61H1Y|  0|160|  9|
|SLpU2LGv54g|  2|415|196|
|IJ24VtLd1jU|  4|186| 44|
|M_WD9Dxayk8|  3|225| 44|
|i7J4C8LHE6Y|  1|144| 26|
+-----------+---+---+---+
only showing top 20 rows



In [16]:
video_sentiment.write.csv("video_sentiment_output.csv", header=True, mode="overwrite")